In [2]:
%pylab inline
import numpy as np
from tqdm import trange
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [3]:
X_train = pd.read_csv("train.csv").drop('label', axis=1).to_numpy()
Y_train = pd.read_csv("train.csv")['label'].to_numpy()

In [4]:
# model
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)
class TestNet(torch.nn.Module):
  def __init__(self):
    super(TestNet, self).__init__()
    self.l1 = nn.Linear(784, 128, bias=False)
    self.l2 = nn.Linear(128, 10, bias=False)
    self.sm = nn.LogSoftmax(dim=1)
  def forward(self, x):
    x = F.relu(self.l1(x))
    x = self.l2(x)
    x = self.sm(x)
    return x

In [5]:
model = TestNet()

In [6]:
loss_function = nn.NLLLoss(reduction='none')
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0)
BS = 128
losses, accuracies = [], []

In [ ]:
for i in (t := trange(1000)):
  samp = np.random.randint(0, X_train.shape[0], size=(BS))
  X = torch.tensor(X_train[samp].reshape((-1, 28*28))).float()
  Y = torch.tensor(Y_train[samp]).long()
  model.zero_grad()
  out = model(X)
  cat = torch.argmax(out, dim=1)
  accuracy = (cat == Y).float().mean()
  loss = loss_function(out, Y)
  loss = loss.mean()
  loss.backward()
  optim.step()
  loss, accuracy = loss.item(), accuracy.item()
  losses.append(loss)
  accuracies.append(accuracy)
  t.set_description("loss %.2f accuracy %.2f" % (loss, accuracy))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
plt.ylim(-0.1, 1.1)
plot(losses)
plot(accuracies)

In [ ]:
# Loading in real test file
testset = pd.read_csv("test.csv")
testset = torch.Tensor(testset.values)
print(testset.shape)

In [ ]:
#Generate submission file 
f = open("submission.txt", "w")
p = []
for i in range(0, 28000):
    pred = model(testset[i].view(-1, 784))
    f.write(str(i+1) + ',' + str(pred.argmax().numpy()) + '\n')